# GPU Sandbox
Use inside an `srun --partition=gputest --gres=gpu:v100:1` shell.

* Kernel: `pf_models` or your active env.
* Verifies CUDA visibility and runs one forward on GPU.


In [ ]:
import os, yaml, torch
os.environ.setdefault('PYTHONPATH', 'models')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

from models.train.core.pf_dataloader import build_dataloaders
from models.train.core.config import DotDict
from models.backbones.registry import build_backbone

cfg_path = 'configs/train/train_smoke.yaml'
cfg = yaml.safe_load(open(cfg_path))

train_loader, _, _ = build_dataloaders(cfg['data'])
batch = next(iter(train_loader))
batch = {k: v.to(device) if hasattr(v, 'to') else v for k, v in batch.items()}
print({k: (v.shape, v.dtype, v.device) for k, v in batch.items()})

model = build_backbone(DotDict(cfg['model'])).to(device)
model.eval()
with torch.no_grad():
    out = model(batch['x'], t=None, cond=batch.get('cond'))
print('output:', out.shape, out.dtype, out.device)
